In [1]:
import os

os.chdir("../..")
import django_init


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# For better printing

from rich import print
from rich.console import Console
from rich.markdown import Markdown
from rich.table import Table

console = Console()
from utils.sql import use_indexes


In [4]:
from utils.perf_display import format_duration, perf_counter
from rest_framework.test import APIClient

client = APIClient()


In [5]:
from books.models import Library, Review, Book
from django.db.models import Count

public = Library.objects.order_by("-id").first()
public_id = public.id

# Get the library with the most books
alexandria_id, book_count = (
    Book.objects.values("library_id")
    .annotate(book_count=Count("id"))
    .order_by("-book_count")
    .values_list("library_id", "book_count")
    .first()
)

alexandria = Library.objects.get(id=alexandria_id)

---

### 3rd example: annotated query

Let's continue with another book endpoint:

This time we want to retrieve paginated list of books, containing some additional information: the list of related tags, count of reviews, and author
A minimal implementation is available in [this file]("../../../../books/views/book/annotate_books_aggregate.py).


In [6]:
with perf_counter(time_sql=True):
    client.get(f"/books/{alexandria_id}/aggregate?ordering=release_date,id")

Duration
  Total: 18.38
  SQL: 18.30

In [7]:
from django.db.models import QuerySet
from django.contrib.postgres.aggregates import ArrayAgg
from books.views.book.list_books_aggregate import serialize_books


def get_queryset(library_id, page_size=20, orderings=("release_date", "id")):
    return (
        Book.objects.filter(library_id=library_id)
        .order_by(*orderings)
        .select_related("author")[:page_size]
    )


def annotate_books_aggregate(book_qs: QuerySet[Book]):
    """
    return a list of book objects,
    annotated with a list of tag names, and count of reviews
    """
    return book_qs.annotate(
        review_count=Count("reviews"), tag_names=ArrayAgg("tags__name")
    )

In [8]:
with perf_counter(time_sql=True, print_sql=True):
    qs = annotate_books_aggregate(get_queryset(alexandria_id))
    result = serialize_books(qs)
print(result[0])


Duration
  Total: 17.61
  SQL: 17.61 

      (duration: 17.606)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", COUNT("books_review"."id") AS "review_count", 
ARRAY_AGG("books_booktag"."name" ) AS "tag_names", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" LEFT OUTER JOIN "books_review" ON ("books_book"."id" 
= "books_review"."book_id") LEFT OUTER JOIN "books_booktag" ON ("books_book"."id" = "books_booktag"."book_id") 
INNER JOIN "books_person" ON ("books_book"."author_id" = "books_person"."id") WHERE "books_book"."library_id" = 6 
GROUP BY "books_book"."id", "books_person"."id" ORDER BY "books_book"."release_date" ASC, "books_book"."id" ASC 
LIMIT 20

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 40,
    'tag_names': [
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'braille',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'comics',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'movie',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio',
        'audio'
    ],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

Do not forget about duplicates when doing multiple aggregations!

Luckily, both `Count` and `ArrayAgg` accept a `distinct` argument


In [9]:
def annotate_books_aggregate(book_qs: QuerySet[Book]):
    return book_qs.annotate(
        review_count=Count("reviews", distinct=True),
        tag_names=ArrayAgg("tags__name", distinct=True),
    )

In [10]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset(alexandria_id)
    qs = annotate_books_aggregate(qs)
    result = serialize_books(qs)
print(result[0])


Duration
  Total: 17.51
  SQL: 17.51 

      (duration: 17.509)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", COUNT(DISTINCT "books_review"."id") AS "review_count", 
ARRAY_AGG(DISTINCT "books_booktag"."name" ) AS "tag_names", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" LEFT OUTER JOIN "books_review" ON ("books_book"."id" 
= "books_review"."book_id") LEFT OUTER JOIN "books_booktag" ON ("books_book"."id" = "books_booktag"."book_id") 
INNER JOIN "books_person" ON ("books_book"."author_id" = "books_person"."id") WHERE "books_book"."library_id" = 6 
GROUP BY "books_book"."id", "books_person"."id" ORDER BY "books_book"."release_date" ASC, "books_book"."id" ASC 
LIMIT 20

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 10,
    'tag_names': ['audio', 'braille', 'comics', 'movie'],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

Now we have a baseline for our endpoint, let's try to improve it.


In [11]:
qs = get_queryset(alexandria_id)
qs = annotate_books_aggregate(qs)
result = qs.explain(analyze=True)

print(result)

Limit  (cost=3653166.13..3653166.18 rows=20 width=912) (actual time=18626.307..18892.615 rows=20 loops=1)
  ->  Sort  (cost=3653166.13..3668016.84 rows=5940286 width=912) (actual time=18273.309..18539.615 rows=20 loops=1)
        Sort Key: books_book.release_date, books_book.id
        Sort Method: top-N heapsort  Memory: 74kB
        ->  GroupAggregate  (cost=545611.43..3495097.25 rows=5940286 width=912) (actual time=4701.367..18400.389 
rows=200711 loops=1)
              Group Key: books_book.id, books_person.id
              ->  Merge Left Join  (cost=545611.43..3361440.82 rows=5940286 width=886) (actual 
time=4619.543..14294.172 rows=6069763 loops=1)
                    Merge Cond: (books_book.id = books_review.book_id)
                    ->  Gather Merge  (cost=545610.99..614738.97 rows=593544 width=878) (actual 
time=4618.033..5313.510 rows=605010 loops=1)
                          Workers Planned: 2
                          Workers Launched: 2
                          ->  Sort  (cost=544610.96..545229.24 rows=247310 width=878) (actual 
time=4581.406..4726.694 rows=201670 loops=3)
                                Sort Key: books_book.id, books_book.author_id
                                Sort Method: external merge  Disk: 172456kB
                                Worker 0:  Sort Method: external merge  Disk: 182464kB
                                Worker 1:  Sort Method: external merge  Disk: 177952kB
                                ->  Parallel Hash Join  (cost=231382.69..331412.98 rows=247310 width=878) (actual 
time=3636.083..3745.313 rows=201670 loops=3)
                                      Hash Cond: (books_book.author_id = books_person.id)
                                      ->  Parallel Hash Left Join  (cost=227885.66..318369.62 rows=247310 width=97)
(actual time=2448.433..3518.788 rows=201670 loops=3)
                                            Hash Cond: (books_book.id = books_booktag.book_id)
                                            ->  Parallel Bitmap Heap Scan on books_book  (cost=2318.39..64220.27 
rows=82471 width=91) (actual time=22.941..108.435 rows=66904 loops=3)
                                                  Recheck Cond: (library_id = 6)
                                                  Heap Blocks: exact=11137
                                                  ->  Bitmap Index Scan on books_book_library_id_9380ba4e  
(cost=0.00..2268.90 rows=197930 width=0) (actual time=15.710..15.711 rows=200711 loops=1)
                                                        Index Cond: (library_id = 6)
                                            ->  Parallel Hash  (cost=138661.23..138661.23 rows=4999523 width=14) 
(actual time=2236.130..2236.130 rows=3999451 loops=3)
                                                  Buckets: 131072  Batches: 256  Memory Usage: 3296kB
                                                  ->  Parallel Seq Scan on books_booktag  (cost=0.00..138661.23 
rows=4999523 width=14) (actual time=197.740..789.655 rows=3999451 loops=3)
                                      ->  Parallel Hash  (cost=2182.24..2182.24 rows=11824 width=781) (actual 
time=10.564..10.565 rows=6700 loops=3)
                                            Buckets: 8192  Batches: 8  Memory Usage: 2144kB
                                            ->  Parallel Seq Scan on books_person  (cost=0.00..2182.24 rows=11824 
width=781) (actual time=0.007..3.107 rows=6700 loops=3)
                    ->  Index Scan using books_review_book_id_a67a4c60 on books_review  (cost=0.44..2585866.27 
rows=40045392 width=16) (actual time=0.078..6345.773 rows=24078361 loops=1)
Planning Time: 2.667 ms
JIT:
  Functions: 66
  Options: Inlining true, Optimization true, Expressions true, Deforming true
  Timing: Generation 11.069 ms, Inlining 178.089 ms, Optimization 472.468 ms, Emission 295.915 ms, Total 957.541 ms
Execution Time: 18931.454 ms

Right, this plan starts to be really complex. Several tools are available online to decrypt PostgreSQL plans, such as [this one](https://explain.dalibo.com/plan/g5h9cad9a38869gc)

We can now see, as expected, that the slow operations are the scan on `books_review` and `books_booktag`.

Even though the planner uses an index, it retrieves almost every row on both those tables, before merging them.

The aggregation is applied only at the end, just before sorting by `release_date`.


In [12]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset(alexandria_id, orderings=("id",))
    qs = annotate_books_aggregate(qs)
    result = serialize_books(qs)

print(result[0])


Duration
  Total: 0.11
  SQL: 0.10 

      (duration: 0.105)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", COUNT(DISTINCT "books_review"."id") AS "review_count", 
ARRAY_AGG(DISTINCT "books_booktag"."name" ) AS "tag_names", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" LEFT OUTER JOIN "books_review" ON ("books_book"."id" 
= "books_review"."book_id") LEFT OUTER JOIN "books_booktag" ON ("books_book"."id" = "books_booktag"."book_id") 
INNER JOIN "books_person" ON ("books_book"."author_id" = "books_person"."id") WHERE "books_book"."library_id" = 6 
GROUP BY "books_book"."id", "books_person"."id" ORDER BY "books_book"."id" ASC LIMIT 20

{
    'id': 1005,
    'title': 'Fear not major top arm social follow economy.',
    'release_date': datetime.date(1968, 10, 1),
    'library_id': 6,
    'review_count': 9,
    'tag_names': ['movie'],
    'author': {'id': 3489, 'name': 'Wesley Gibbs'}
}

In [13]:
qs = get_queryset(alexandria_id, orderings=("id",))
qs = annotate_books_aggregate(qs)
result = qs.explain(analyze=True)

print(result)

Limit  (cost=28.33..46.88 rows=20 width=912) (actual time=37.564..40.077 rows=20 loops=1)
  ->  GroupAggregate  (cost=28.33..5509789.13 rows=5940286 width=912) (actual time=37.562..40.070 rows=20 loops=1)
        Group Key: books_book.id, books_person.id
        ->  Incremental Sort  (cost=28.33..5376132.69 rows=5940286 width=886) (actual time=37.500..39.524 rows=489 
loops=1)
              Sort Key: books_book.id, books_person.id
              Presorted Key: books_book.id
              Full-sort Groups: 12  Sort Method: quicksort  Average Memory: 77kB  Peak Memory: 83kB
              Pre-sorted Groups: 5  Sort Method: quicksort  Average Memory: 39kB  Peak Memory: 40kB
              ->  Merge Left Join  (cost=1.60..5045795.50 rows=5940286 width=886) (actual time=1.503..38.821 
rows=525 loops=1)
                    Merge Cond: (books_book.id = books_review.book_id)
                    ->  Nested Loop Left Join  (cost=1.16..2299093.65 rows=593544 width=878) (actual 
time=0.368..0.802 rows=59 loops=1)
                          ->  Nested Loop  (cost=0.73..1123055.75 rows=197930 width=872) (actual time=0.342..0.640 
rows=23 loops=1)
                                ->  Index Scan using books_book_pkey on books_book  (cost=0.43..227962.47 
rows=197930 width=91) (actual time=0.310..0.401 rows=23 loops=1)
                                      Filter: (library_id = 6)
                                      Rows Removed by Filter: 1225
                                ->  Memoize  (cost=0.30..5.24 rows=1 width=781) (actual time=0.009..0.009 rows=1 
loops=23)
                                      Cache Key: books_book.author_id
                                      Cache Mode: logical
                                      Hits: 0  Misses: 23  Evictions: 0  Overflows: 0  Memory Usage: 19kB
                                      ->  Index Scan using books_person_pkey on books_person  (cost=0.29..5.23 
rows=1 width=781) (actual time=0.005..0.005 rows=1 loops=23)
                                            Index Cond: (id = books_book.author_id)
                          ->  Index Scan using books_booktag_book_id_361012b4 on books_booktag  (cost=0.43..5.85 
rows=9 width=14) (actual time=0.004..0.005 rows=3 loops=23)
                                Index Cond: (book_id = books_book.id)
                    ->  Index Scan using books_review_book_id_a67a4c60 on books_review  (cost=0.44..2585866.27 
rows=40045392 width=16) (actual time=0.027..34.091 rows=25725 loops=1)
Planning Time: 2.179 ms
Execution Time: 40.479 ms

By comparison, we can look at [the plan](https://explain.dalibo.com/plan/31ae7aa178g5f1da) when ordering by `id`. It's completely different.
1 key difference is the sort, which is now incremental

> Compared to regular sorts, sorting incrementally allows returning tuples before the entire result set has been sorted, which particularly enables optimizations with LIMIT queries. It may also reduce memory usage and the likelihood of spilling sorts to disk, but it comes at the cost of the increased overhead of splitting the result set into multiple sorting batches.

<br>
<br>


---

While we could try to find clever indexes to improve our queries, we'll try to find other solutions to reach the same result.


<br>
<br>
Since the aggregates are expensive, and only used for the output, but not to filter book rows, we can try to split the logic in two parts


In [14]:
def get_book_ids(library_id, page_size=20, orderings=("release_date", "id")):
    return list(
        Book.objects.filter(library_id=library_id)
        .order_by(*orderings)[:page_size]
        .values_list("id", flat=True)
    )


def get_queryset_from_ids(book_ids, orderings=("release_date", "id")):
    return (
        Book.objects.filter(id__in=book_ids)
        .select_related("author")
        .order_by(*orderings)
    )


def get_queryset_using_ids(library_id, page_size=20, orderings=("release_date", "id")):
    book_ids = get_book_ids(library_id, page_size=page_size, orderings=orderings)
    return get_queryset_from_ids(book_ids, orderings=orderings)

In [15]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset_using_ids(alexandria_id)
    qs = annotate_books_aggregate(qs)

    result = serialize_books(qs)
print(result[0])

Duration
  Total: 0.42
  SQL: 0.41 

      (duration: 0.238)   SELECT "books_book"."id" FROM "books_book" WHERE "books_book"."library_id" = 6 ORDER BY 
"books_book"."release_date" ASC, "books_book"."id" ASC LIMIT 20 

      (duration: 0.177)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", COUNT(DISTINCT "books_review"."id") AS "review_count", 
ARRAY_AGG(DISTINCT "books_booktag"."name" ) AS "tag_names", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" LEFT OUTER JOIN "books_review" ON ("books_book"."id" 
= "books_review"."book_id") LEFT OUTER JOIN "books_booktag" ON ("books_book"."id" = "books_booktag"."book_id") 
INNER JOIN "books_person" ON ("books_book"."author_id" = "books_person"."id") WHERE "books_book"."id" IN (251127, 
359638, 1156714, 1213190, 1402817, 303340, 645062, 712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 
278596, 710710, 17364, 76125, 148950, 420567) GROUP BY "books_book"."id", "books_person"."id" ORDER BY 
"books_book"."release_date" ASC, "books_book"."id" ASC

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 10,
    'tag_names': ['audio', 'braille', 'comics', 'movie'],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

In [16]:
qs = get_queryset_using_ids(alexandria_id)
qs = annotate_books_aggregate(qs)
result = qs.explain(analyze=True)

print(result)

Sort  (cost=1840.63..1842.13 rows=600 width=912) (actual time=170.571..170.574 rows=20 loops=1)
  Sort Key: books_book.release_date, books_book.id
  Sort Method: quicksort  Memory: 51kB
  ->  GroupAggregate  (cost=91.06..1812.94 rows=600 width=912) (actual time=0.372..170.515 rows=20 loops=1)
        Group Key: books_book.id, books_person.id
        ->  Incremental Sort  (cost=91.06..1799.44 rows=600 width=886) (actual time=0.317..155.780 rows=20662 
loops=1)
              Sort Key: books_book.id, books_person.id
              Presorted Key: books_book.id
              Full-sort Groups: 12  Sort Method: quicksort  Average Memory: 109kB  Peak Memory: 115kB
              Pre-sorted Groups: 5  Sort Methods: quicksort, external merge  Average Memory: 58kB  Peak Memory: 
79kB  Average Disk: 2665kB  Peak Disk: 13328kB
              ->  Nested Loop Left Join  (cost=1.60..1766.08 rows=600 width=886) (actual time=0.132..94.851 
rows=20662 loops=1)
                    ->  Nested Loop Left Join  (cost=1.16..1558.89 rows=200 width=880) (actual time=0.099..74.596 
rows=20281 loops=1)
                          ->  Nested Loop  (cost=0.72..251.46 rows=20 width=872) (actual time=0.071..0.576 rows=20 
loops=1)
                                ->  Index Scan using books_book_pkey on books_book  (cost=0.43..93.36 rows=20 
width=91) (actual time=0.042..0.343 rows=20 loops=1)
                                      Index Cond: (id = ANY 
('{251127,359638,1156714,1213190,1402817,303340,645062,712181,912196,1345919,344098,821709,1976681,183742,278596,71
0710,17364,76125,148950,420567}'::bigint[]))
                                ->  Index Scan using books_person_pkey on books_person  (cost=0.29..7.90 rows=1 
width=781) (actual time=0.010..0.010 rows=1 loops=20)
                                      Index Cond: (id = books_book.author_id)
                          ->  Index Scan using books_review_book_id_a67a4c60 on books_review  (cost=0.44..61.99 
rows=338 width=16) (actual time=0.016..3.373 rows=1014 loops=20)
                                Index Cond: (book_id = books_book.id)
                    ->  Memoize  (cost=0.45..8.73 rows=9 width=14) (actual time=0.000..0.000 rows=1 loops=20281)
                          Cache Key: books_book.id
                          Cache Mode: logical
                          Hits: 20261  Misses: 20  Evictions: 0  Overflows: 0  Memory Usage: 5kB
                          ->  Index Scan using books_booktag_book_id_361012b4 on books_booktag  (cost=0.43..8.72 
rows=9 width=14) (actual time=0.014..0.016 rows=3 loops=20)
                                Index Cond: (book_id = books_book.id)
Planning Time: 1.656 ms
Execution Time: 174.037 ms

Another try: Instead of using aggregation (with a `GROUP BY` clause), can use subqueries.


In [17]:
from django.contrib.postgres.expressions import ArraySubquery
from django.db.models import Count, OuterRef, QuerySet
from django.db.models.functions import Coalesce

from books.models import Book, BookTag, Review


def annotate_books_subquery(book_qs: QuerySet[Book]):
    return book_qs.annotate(
        review_count=Coalesce(
            Review.objects.filter(book_id=OuterRef("id"))
            .values("book_id")
            .annotate(count=Count("id", distinct=True))
            .values("count"),
            0,
        ),
        tag_names=ArraySubquery(
            BookTag.objects.filter(book_id=OuterRef("id")).values("name")
        ),
    )


In [18]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset_using_ids(alexandria_id)
    qs = annotate_books_subquery(qs)

    result = serialize_books(qs)

print(result[0])

Duration
  Total: 0.33
  SQL: 0.32 

      (duration: 0.253)   SELECT "books_book"."id" FROM "books_book" WHERE "books_book"."library_id" = 6 ORDER BY 
"books_book"."release_date" ASC, "books_book"."id" ASC LIMIT 20 

      (duration: 0.067)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", COALESCE((SELECT COUNT(DISTINCT U0."id") AS "count" FROM 
"books_review" U0 WHERE U0."book_id" = ("books_book"."id") GROUP BY U0."book_id"), 0) AS "review_count", 
ARRAY(SELECT U0."name" FROM "books_booktag" U0 WHERE U0."book_id" = ("books_book"."id")) AS "tag_names", 
"books_person"."id", "books_person"."email", "books_person"."name", "books_person"."bio" FROM "books_book" INNER 
JOIN "books_person" ON ("books_book"."author_id" = "books_person"."id") WHERE "books_book"."id" IN (251127, 359638,
1156714, 1213190, 1402817, 303340, 645062, 712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 278596, 
710710, 17364, 76125, 148950, 420567) ORDER BY "books_book"."release_date" ASC, "books_book"."id" ASC

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 10,
    'tag_names': ['movie', 'audio', 'comics', 'braille'],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

In [19]:
qs = get_queryset_using_ids(alexandria_id)
qs = annotate_books_subquery(qs)
result = qs.explain(analyze=True)

print(result)

Sort  (cost=1171.42..1171.47 rows=20 width=912) (actual time=75.773..75.776 rows=20 loops=1)
  Sort Key: books_book.release_date, books_book.id
  Sort Method: quicksort  Memory: 51kB
  ->  Nested Loop  (cost=0.72..1170.99 rows=20 width=912) (actual time=0.150..75.718 rows=20 loops=1)
        ->  Index Scan using books_book_pkey on books_book  (cost=0.43..93.36 rows=20 width=91) (actual 
time=0.039..0.281 rows=20 loops=1)
              Index Cond: (id = ANY 
('{251127,359638,1156714,1213190,1402817,303340,645062,712181,912196,1345919,344098,821709,1976681,183742,278596,71
0710,17364,76125,148950,420567}'::bigint[]))
        ->  Index Scan using books_person_pkey on books_person  (cost=0.29..7.90 rows=1 width=781) (actual 
time=0.009..0.009 rows=1 loops=20)
              Index Cond: (id = books_book.author_id)
        SubPlan 1
          ->  GroupAggregate  (cost=0.44..37.26 rows=338 width=16) (actual time=3.741..3.741 rows=1 loops=20)
                Group Key: u0.book_id
                ->  Index Scan using books_review_book_id_a67a4c60 on books_review u0  (cost=0.44..32.19 rows=338 
width=16) (actual time=0.015..3.206 rows=1014 loops=20)
                      Index Cond: (book_id = books_book.id)
        SubPlan 2
          ->  Index Scan using books_booktag_book_id_361012b4 on books_booktag u0_1  (cost=0.43..8.72 rows=9 
width=6) (actual time=0.015..0.016 rows=3 loops=20)
                Index Cond: (book_id = books_book.id)
Planning Time: 0.696 ms
Execution Time: 75.889 ms

This method is already really fast.k
1 thing to note though: the `loops=20` on the `books_reviews` index.

If the query becomes more complex, we'll have to execute this part once par item in the page

Let's try to avoid this loop, using multiple queries, and joining them in python


In [20]:
def annotate_books_python(books):
    book_ids = [book.id for book in books]

    review_count = dict(
        Review.objects.filter(book_id__in=book_ids)
        .values("book_id")
        .annotate(count=Count("id"))
        .values_list("book_id", "count")
    )
    tag_list = dict(
        BookTag.objects.filter(book_id__in=book_ids)
        .values("book_id")
        .annotate(tags=ArrayAgg("name"))
        .values_list("book_id", "tags")
    )
    books = list(qs)
    for book in books:
        book.review_count = review_count[book.id]
        book.tag_names = tag_list.get(book.id, [])
    return books

In [21]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset_using_ids(alexandria_id, page_size=20)
    qs = annotate_books_python(list(qs))

    result = serialize_books(qs)
print(result[0])

Duration
  Total: 0.31
  SQL: 0.30 

      (duration: 0.226)   SELECT "books_book"."id" FROM "books_book" WHERE "books_book"."library_id" = 6 ORDER BY 
"books_book"."release_date" ASC, "books_book"."id" ASC LIMIT 20 

      (duration: 0.003)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" INNER JOIN "books_person" ON 
("books_book"."author_id" = "books_person"."id") WHERE "books_book"."id" IN (251127, 359638, 1156714, 1213190, 
1402817, 303340, 645062, 712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 
148950, 420567) ORDER BY "books_book"."release_date" ASC, "books_book"."id" ASC 

      (duration: 0.072)   SELECT "books_review"."book_id", COUNT("books_review"."id") AS "count" FROM 
"books_review" WHERE "books_review"."book_id" IN (251127, 359638, 1156714, 1213190, 1402817, 303340, 645062, 
712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 148950, 420567) GROUP BY 
"books_review"."book_id" 

      (duration: 0.001)   SELECT "books_booktag"."book_id", ARRAY_AGG("books_booktag"."name" ) AS "tags" FROM 
"books_booktag" WHERE "books_booktag"."book_id" IN (251127, 359638, 1156714, 1213190, 1402817, 303340, 645062, 
712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 148950, 420567) GROUP BY 
"books_booktag"."book_id"

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 10,
    'tag_names': ['movie', 'audio', 'comics', 'braille'],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

That's a bit faster.

---

We can try to reduce the number of queries by using Common Table Expressions (CTE), i.e. queries having `WITH` clauses.


In [22]:
raw_query = """
WITH review_count AS (
    SELECT
        books_review.book_id as id,
        count(books_review.id) as review_count
    FROM
        books_review
    WHERE
        books_review.book_id IN %(book_ids)s
        GROUP BY
            book_id
),
tag_names AS (
    SELECT
        books_booktag.book_id as id,
        array_agg(DISTINCT books_booktag.name) as tags_list
    FROM
        books_booktag
    WHERE
        books_booktag.book_id IN %(book_ids)s
        GROUP BY
            book_id
)
SELECT
    id,
    review_count,
    COALESCE(tags_list, '{}') as tags_list
FROM
    review_count FULL OUTER JOIN tag_names USING (id);

"""


def annotate_books_cte(books):
    book_ids = tuple(book.id for book in books)
    annotations_by_book_id = {
        raw_book.id: {
            "review_count": raw_book.review_count,
            "tag_names": raw_book.tags_list,
        }
        for raw_book in Book.objects.raw(raw_query, params={"book_ids": book_ids})
    }
    for book in books:
        annotations = annotations_by_book_id[book.id]
        for name, value in annotations.items():
            setattr(book, name, value)
    return books


In [23]:
with perf_counter(time_sql=True, print_sql=True):
    qs = get_queryset_using_ids(alexandria_id, page_size=20)
    qs = annotate_books_cte(list(qs))

    result = serialize_books(qs)
print(result[0])

Duration
  Total: 0.28
  SQL: 0.27 

      (duration: 0.188)   SELECT "books_book"."id" FROM "books_book" WHERE "books_book"."library_id" = 6 ORDER BY 
"books_book"."release_date" ASC, "books_book"."id" ASC LIMIT 20 

      (duration: 0.006)   SELECT "books_book"."id", "books_book"."title", "books_book"."author_id", 
"books_book"."release_date", "books_book"."library_id", "books_person"."id", "books_person"."email", 
"books_person"."name", "books_person"."bio" FROM "books_book" INNER JOIN "books_person" ON 
("books_book"."author_id" = "books_person"."id") WHERE "books_book"."id" IN (251127, 359638, 1156714, 1213190, 
1402817, 303340, 645062, 712181, 912196, 1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 
148950, 420567) ORDER BY "books_book"."release_date" ASC, "books_book"."id" ASC 

      (duration: 0.078)   
WITH review_count AS (
    SELECT
        books_review.book_id as id,
        count(books_review.id) as review_count
    FROM
        books_review
    WHERE
        books_review.book_id IN (251127, 359638, 1156714, 1213190, 1402817, 303340, 645062, 712181, 912196, 
1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 148950, 420567)
        GROUP BY
            book_id
),
tag_names AS (
    SELECT
        books_booktag.book_id as id,
        array_agg(DISTINCT books_booktag.name) as tags_list
    FROM
        books_booktag
    WHERE
        books_booktag.book_id IN (251127, 359638, 1156714, 1213190, 1402817, 303340, 645062, 712181, 912196, 
1345919, 344098, 821709, 1976681, 183742, 278596, 710710, 17364, 76125, 148950, 420567)
        GROUP BY
            book_id
)
SELECT
    id,
    review_count,
    COALESCE(tags_list, '{}') as tags_list
FROM
    review_count FULL OUTER JOIN tag_names USING (id);

{
    'id': 251127,
    'title': 'Design care there six must keep on.',
    'release_date': datetime.date(1923, 5, 31),
    'library_id': 6,
    'review_count': 10,
    'tag_names': ['audio', 'braille', 'comics', 'movie'],
    'author': {'id': 3454, 'name': 'Charles Ingram'}
}

In [29]:
from rich.table import Table
import time
from utils.perf_display import format_duration

table = Table(show_lines=True)
table.add_column("name", style="bold green")
table.add_column("duration")


def add_row(annotate_func):
    start = time.perf_counter()
    qs = get_queryset_using_ids(alexandria_id, page_size=100)
    qs = annotate_func(qs)
    serialize_books(qs)
    duration = time.perf_counter() - start
    table.add_row(annotate_func.__name__, format_duration(duration))


for annotate_func in [
    annotate_books_aggregate,
    annotate_books_subquery,
    annotate_books_python,
    annotate_books_cte,
]:
    add_row(annotate_func)

console.print(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ name                     ┃ duration ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ annotate_books_aggregate │ 0.56     │
├──────────────────────────┼──────────┤
│ annotate_books_subquery  │ 0.44     │
├──────────────────────────┼──────────┤
│ annotate_books_python    │ 0.26     │
├──────────────────────────┼──────────┤
│ annotate_books_cte       │ 0.22     │
└──────────────────────────┴──────────┘

We could even merge this query with the one retrieving all the `Book` and `Person` fields.

This would require to write more raw SQL code, though, which is not always a good idea
